# DASH & PLOTLY

In [ ]:
# Import required libraries
import pandas as pd
import plotly.graph_objects as go
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px
file = r"spacex_launch_dash.csv"
df = pd.read_csv(file)
min_value = df["Payload Mass (kg)"].min()
max_value = df["Payload Mass (kg)"].max()
booster_version = set(pd.Series(df["Booster Version"].unique()).map(lambda x: x.split()[1]))
app = dash.Dash(__name__)
app.layout = html.Div(children=[
    html.H1(children='SpaceX'),
    #Task 1: add dropdown
    dcc.Dropdown(id='dropdown_id',
                options=[
                    {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                    {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                    {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                    {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                    {'label': 'ALL', 'value': 'ALL'},
                ],
                value='ALL',
                placeholder="Selec a launch site here",
                searchable=True
                ),
    #Task 2: pie plot
                html.Div(dcc.Graph(id='pie-plot', style={'width': 800, 'height': 800})),
    #TASK 3: Add a Range Slider to Select Payload
    dcc.RangeSlider(id='payload-slider',
                min=0, max=10000, step=1000,
                value=[min_value, max_value]),
    #TASK 4: Add a callback function to render the success-payload-scatter-chart scatter plot
    html.Div(dcc.Graph(id='cat-plot', style={'width': 800, 'height': 800})),
])
@app.callback(
    [Output(component_id='pie-plot', component_property='figure'),
     Output(component_id='cat-plot', component_property='figure')],
    [Input(component_id='dropdown_id', component_property='value'),
    Input(component_id="payload-slider", component_property="value")])
def get_graph(selected_site,payload):
    #df_filter =  df[df["Launch Site"]==selected_site]
    #The pie_chart
    if selected_site == 'ALL':
        filtered_df = pd.DataFrame(df.groupby('Launch Site').count()).reset_index()[["Launch Site","class"]]
        fig_pie = px.pie(filtered_df, values='class', names='Launch Site', hole=.3)
        #The pie plot
        df_cat = df[(df["Payload Mass (kg)"] >= payload[0]) & (df["Payload Mass (kg)"] <= payload[1])]
        df_cat["Booster"] = df["Booster Version"].map(lambda x: x.split()[1])
        fig_cat = px.scatter(df_cat, x="Payload Mass (kg)", y="class",color="Booster")
    else:
        filtered_df = df[df["Launch Site"] == selected_site]
        filtered_df = pd.DataFrame(filtered_df.groupby('class')['Launch Site'].count()).reset_index()
        fig_pie = px.pie(filtered_df, values='Launch Site', names='class', hole=.3)
        #The scatter plot
        df_cat = df[df["Launch Site"]==selected_site]
        df_cat["Booster"] = df["Booster Version"].map(lambda x: x.split()[1])
        df_cat = df_cat[(df_cat["Payload Mass (kg)"] >= payload[0]) & (df_cat["Payload Mass (kg)"] <= payload[1])]
        fig_cat = px.scatter(df_cat, x="Payload Mass (kg)", y="class",color="Booster")
        
        
    return fig_pie,fig_cat
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Jul/2023 19:00:20] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Jul/2023 19:00:21] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [14/Jul/2023 19:00:21] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [14/Jul/2023 19:00:21] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Jul/2023 19:00:21] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Jul/2023 19:00:21] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Jul/2023 19:00:21] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Jul/2023 19:00:26] "POST /_dash-update-component HTTP/1.1" 200 -
C:\Users\THUY TRAN\AppData\Local\Temp\ipykernel_3372\2169497287.py:60: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

# Finding Insights Visually
<h2>Now with the dashboard completed, you should be able to use it to analyze SpaceX launch data, and answer the following questions:</h2>
<ol>
    <li>Which site has the largest successful launches? <span style="color: red;">=> CCAFS LC-40</span></li>
    <li>Which site has the highest launch success rate?
    <span style="color: red;">=> CCAFS SLC-40</span></li>
    <li>Which payload range(s) has the highest launch success rate?
    <span style="color: red;">=> 3000-4000</span></li>
    <li>Which payload range(s) has the lowest launch success rate?=> 0-2000</li>
    <li>Which F9 Booster version (v1.0, v1.1, FT, B4, B5, etc.) has the highest
    launch success rate?<span style="color: red;">=> FT</span></li>
</ol>